### Prepare Data for Crowdsourcing

In [ ]:
!pip install -q nlproc_tools

In [ ]:
from nlproc_tools import load_interface

In [ ]:
# Let's begin by downloading the demo typology and data
!curl -so crowdsource.yml https://nlproc.tools/templates/demo/crowdsource.yml
!curl -so crowdsource.json https://nlproc.tools/data/demo/crowdsource.json

In [ ]:
# Studies use a unique completion code. We begin by pasting the code into
# the interface template.
completion_code = "YOUR_COMPLETION_CODE"

In [ ]:
# Now we can load and customize our interface template
Interface = load_interface("crowdsource.yml")
print(Interface.template_label)
print(Interface.template_description)

Interface.prolific_completion_code = completion_code

In [ ]:
# TOOD: Custom deployment link!

In [ ]:
template_path = "davidheineman/nlproc.tools/main/public/templates/demo/start.yml"
data_path = "davidheineman/nlproc.tools/main/public/data/demo/start.json"

study_url = f"https://nlproc.tools/?gh={template_path}&d={data_path}"
study_url = study_url + "&PROLIFIC_PID={%PROLIFIC_PID%}&STUDY_ID={%STUDY_ID%}&SESSION_ID={%SESSION_ID%}"

print(f"Study will be linked to: {study_url}")

### Deploy to Prolific
For details on the Prolific API, see [**docs.prolific.co/docs/api-docs/public**](https://docs.prolific.co/docs/api-docs/public).

In [ ]:
# We will use Dallinger, a popular annotation service management tool
# Dallinger also has integrations with other services, such as MTurk
!pip install -q dallinger

In [ ]:
# See more usage details at https://github.com/Dallinger/Dallinger/blob/master/dallinger/prolific.py
from dallinger.prolific import ProlificService

In [ ]:
# To get a developer token, visit your prolific workspace -> settings -> API tokens
YOUR_PROLIFIC_API_TOKEN = "YOUR_API_TOKEN" 

# This will read the API token from a local file
import os
if os.path.exists('.prolific_token'):
    YOUR_PROLIFIC_API_TOKEN = open('.prolific_token', "r").read()

prolific = ProlificService(
    api_token=YOUR_PROLIFIC_API_TOKEN,
    api_version="v1",
    referer_header=None
)

In [ ]:
# Prolific annotations are managed by workspaces and grouped by projects. 
response = prolific._req(
    method="GET",
    endpoint=f"/workspaces/",
)
workspace_id = response['results'][0]['id']
print(f"Found {response['meta']['count']} workspace(s), using {response['results'][0]['title']} with id {workspace_id}")

In [ ]:
# Create a new project for our demo annoations.
response = prolific._req(
    method="POST",
    endpoint=f"/workspaces/{workspace_id}/projects/",
    json={
        "title": "My Demo nlproc.tools Project"
    },
)
project_id = response['id']
print(f"Created new project with id {project_id}")

In [ ]:
# Create a new draft study for our project.

# For more information about survey options, refer to:
#   https://docs.prolific.co/docs/api-docs/public/#tag/Studies/paths/~1api~1v1~1studies~1/post
response = prolific.draft_study(
    # Information about the study
    name="My Demo Annotation Task",
    internal_name="demo-annotation-task",
    description="""In this task, you will use nlproc.tools to perform fine-grained evaluation!""",
    # project=project_id,

    # Parameters for your data collection
    estimated_completion_time=3,
    maximum_allowed_time=10,
    reward=2,
    total_available_places=1,
    eligibility_requirements=[],

    # Configuration for our nlproc.tools setup
    external_study_url=study_url,
    completion_code=completion_code,
    completion_option="url",
    prolific_id_option="url_parameters",
    device_compatibility=["desktop"],
    peripheral_requirements=[],
)
study_id = response['id']

print(f"Created draft study with name {response['name']} and ID {study_id}.")

### View your study
Your draft study is now public! To view, visit your Prolific dashboard at [**app.prolific.co/researcher/workspaces**](https://app.prolific.co/researcher/workspaces).

### Manage Existing Studies

In [ ]:
# We can add more slots to an existing study
response = prolific.add_participants_to_study(
    study_id=study_id,
    number_to_add=5
)

print(response)

In [ ]:
# Note: when you are ready, this will make the study publically available to participants
# prolific.publish_study(study_id)

In [ ]:
prolific.get_hits()

In [ ]:
# Cleanup our resources
response = prolific.delete_study(study_id)
if response: print(f"Successfully deleted study: {study_id}!")

response = prolific._req(
    method="PATCH",
    endpoint=f"/projects/{project_id}/",
    json={},
)
if response: print(f"Successfully deleted project {project_id}!")

### Advanced features
Refer to the [**Dallinger library**](https://github.com/Dallinger/Dallinger/blob/master/dallinger/prolific.py) for details about the `ProlificService` class. Here are a few other useful commands:

- `pay_session_bonus` - Pay a worker a bonus
- `approve_participant_session` / `get_participant_session` - Get / approve an annotation

Additionally, you can create your own API calls to Prolific using the format:
```python
response = prolific._req(
    method="XXX",               # The HTTP method, e.g., GET, POST
    endpoint=f"/XXX/YYY/",      # The API endpoint
    json={},                    # The request body in JSON format
)
```
For a full list of API calls and options, please refer to [**docs.prolific.co/docs/api-docs/public**](https://docs.prolific.co/docs/api-docs/public).